# How to use Late Chunk in RAG

Based on the [Late Chunking: Contextual Chunk Embeddings Using Long-Context Embedding Models](https://arxiv.org/abs/2409.04701) paper and the [Late Chunking](https://jina.ai/news/late-chunking-in-long-context-embedding-models) blog post

This notebooks explains how the `Late chunk Embedding` support with `LangChain`.

**Notes:**

- Can combine with any `text splitting` used in LangChain or you can custom with the [Chunk](https://github.com/jina-ai/late-chunking/blob/main/chunked_pooling/chunking.py) used in the paper.

- Support by Late Chunk Qdrant vectorstore

In [ ]:
from qdrant_client import QdrantClient

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embedding import JinaLateChunkEmbeddings
from langchain_community.vectorstore import LateChunkQdrant

from langchain_text_splitters import CharacterTextSplitter

In [ ]:
ROOT = 'demo-qdrant'

# Split
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator="\n\n")

# Text embedding
text_embeddings = JinaLateChunkEmbeddings(jina_api_key="jina_*", model_name="jina-embeddings-v3")

In [ ]:
def init_vectorstore(text_embeddings, text_splitter, collection_name='latechunk', topK=5):
    client = QdrantClient()
    vectorstore = LateChunkQdrant(
        client, collection_name=collection_name,
        embeddings=text_embeddings, text_splitter=text_splitter
    )
    
    if os.path.isdir(os.path.join(ROOT, 'collection', collection_name)):
        print(f"===== Load exits collection: {collection_name} ======")
        vectorstore = vectorstore.from_existing_collection(
            embedding=text_embeddings, path=ROOT, 
            collection_name=collection_name, text_splitter=text_splitter
        )
        
    else:
        print(f"===== Create new collection: {collection_name} ======")
        # Load blog post
        loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
        data = loader.load()

        vectorstore = vectorstore.from_documents(
            documents=data, embedding=text_embeddings, text_splitter=text_splitter,
            path=ROOT, collection_name=collection_name
        )

    return vectorstore.as_retriever(search_kwargs={"k": topK})

vectordb = init_vectorstore(text_embeddings, text_splitter, collection_name='test')

In [ ]:
query = "What are the approaches to Task Decomposition?"

results = vectordb.invoke(query)
len(results)